<a href="https://colab.research.google.com/github/sumanyurosha/tensorflow-specialization/blob/master/Practicing_Text_Generation_with_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import time

# **1. Downloading the Dataset**

In [88]:
path_to_file = keras.utils.get_file("shakespeare.txt", "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt")

text_data = open(path_to_file, mode="rb", ).read().decode(encoding="utf-8")
print("Characters in dataset : {}".format(len(text_data)))

Characters in dataset : 1115394


In [89]:
print(text_data[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



# **2. Data Preprocessing**

In [115]:
vocab = sorted(set(text_data))
print("There are {} unique characters in text".format(len(vocab)))

There are 65 unique characters in text


In [117]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text_data])
print(text_as_int[:5])

[18 47 56 57 58]


In [118]:
for char, _ in zip(char2idx, range(20)):
    print("{}: {}".format(repr(char), char2idx[char]))

'\n': 0
' ': 1
'!': 2
'$': 3
'&': 4
"'": 5
',': 6
'-': 7
'.': 8
'3': 9
':': 10
';': 11
'?': 12
'A': 13
'B': 14
'C': 15
'D': 16
'E': 17
'F': 18
'G': 19


In [121]:
print(text_data[:13])
print(text_as_int[:13])

First Citizen
[18 47 56 57 58  1 15 47 58 47 64 43 52]


In [136]:
seq_length = 100
batch_size = 64
vocab_size = len(char2idx)
embedding_dim = 256

In [137]:
dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
dataset = dataset.batch(seq_length+1, drop_remainder=True)
dataset

<BatchDataset shapes: (101,), types: tf.int64>

In [138]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    output_text = chunk[1:]
    return input_text, output_text

dataset = dataset.map(split_input_target)

In [139]:
for input_text, output_text in dataset.take(3):
    print(repr("".join([idx2char[c] for c in input_text])))
    print(repr("".join([idx2char[c] for c in output_text])))
    print()

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '

'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you '
're all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'

"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us k"
"ow Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"



In [140]:
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

In [141]:
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [142]:
for input, target in dataset.take(1):
    print(input)
    print(target)

tf.Tensor(
[[58 46 63 ... 53 59 50]
 [53 49  1 ...  0  0 16]
 [49  1 63 ...  1 53 59]
 ...
 [51 43 56 ... 59 52 42]
 [ 0 17 52 ...  1 58 46]
 [21 26 21 ...  1 44 53]], shape=(64, 100), dtype=int64)
tf.Tensor(
[[46 63  1 ... 59 50 42]
 [49  1 53 ...  0 16 33]
 [ 1 63 53 ... 53 59 58]
 ...
 [43 56  1 ... 52 42 11]
 [17 52 44 ... 58 46 43]
 [26 21 33 ... 44 53 56]], shape=(64, 100), dtype=int64)


# **Creating a Model**

In [143]:
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy( labels, logits,
                                                        from_logits=True,)


In [157]:
def build_model(batch_size, rnn_units):
    model = keras.models.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim,
                               batch_input_shape=[batch_size, None]),
        keras.layers.GRU(rnn_units, return_sequences=True,
                         stateful=True,
                         recurrent_initializer="glorot_uniform"),
        keras.layers.Dense(vocab_size)
    ])

    return model

In [145]:
checkpoint_dir = "/checkpoints"
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = keras.callbacks.ModelCheckpoint(checkpoint_prefix, 
                                             save_weights_only=True)

In [146]:
for input_example, output_example in dataset.take(1):
    predicted_example = model(input_example)

print(predicted_example.shape)

(64, 100, 65)


In [160]:
model = build_model(batch_size=batch_size, rnn_units=1024)
model.compile(loss=loss, optimizer="adam")
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (64, None, 256)           16640     
_________________________________________________________________
gru_14 (GRU)                 (64, None, 1024)          3938304   
_________________________________________________________________
dense_14 (Dense)             (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [161]:

model.fit(dataset, epochs=30, callbacks=[checkpoint_callback], verbose=1)

Epoch 1/30
172/172 [==============================] - 10s 57ms/step - loss: 2.6601
Epoch 2/30
172/172 [==============================] - 10s 60ms/step - loss: 1.9612
Epoch 3/30
172/172 [==============================] - 10s 61ms/step - loss: 1.6950
Epoch 4/30
172/172 [==============================] - 10s 60ms/step - loss: 1.5469
Epoch 5/30
172/172 [==============================] - 10s 60ms/step - loss: 1.4578
Epoch 6/30
172/172 [==============================] - 10s 58ms/step - loss: 1.3971
Epoch 7/30
172/172 [==============================] - 10s 58ms/step - loss: 1.3499
Epoch 8/30
172/172 [==============================] - 10s 59ms/step - loss: 1.3123
Epoch 9/30
172/172 [==============================] - 10s 59ms/step - loss: 1.2773
Epoch 10/30
172/172 [==============================] - 10s 59ms/step - loss: 1.2436
Epoch 11/30
172/172 [==============================] - 10s 59ms/step - loss: 1.2126
Epoch 12/30
172/172 [==============================] - 10s 60ms/step - loss: 1.1796
E

In [166]:
for input_example, output_example in dataset.take(1):
    predicted_example = model(input_example)

print(predicted_example.shape)

(64, 100, 65)


In [167]:
prediction = tf.random.categorical(predicted_example[0], num_samples=1)
prediction = tf.squeeze(prediction, axis=-1)
prediction

<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([ 1, 58, 52,  1, 53, 43, 57, 41, 53, 52, 53, 59, 56,  1, 42,  1, 63,
       63,  1, 58,  1, 41, 39, 52,  1, 41, 43, 47, 50, 46,  1, 45, 50, 43,
       56,  1, 33, 50, 53, 44, 43, 57, 57,  5, 42,  1, 58, 53,  1, 46, 39,
       51,  6,  1, 61, 46, 43,  1,  1,  5, 43, 57,  1, 39, 43, 54, 43, 56,
       45, 43, 57,  1, 61, 59, 57, 58,  0, 21, 52,  1, 40, 46, 43, 58,  1,
       46, 53, 50, 39, 39, 42, 43,  6, 40, 53, 56, 43,  1, 40, 47])>

In [168]:
prediction = "".join([idx2char[idx] for idx in prediction])
print(prediction)

 tn oesconour d yy t can ceilh gler Ulofess'd to ham, whe  'es aeperges wust
In bhet holaade,bore bi


In [169]:
tf.train.latest_checkpoint(checkpoint_dir)

'/checkpoints/ckpt_30'

In [171]:
model = build_model(batch_size=1, rnn_units=1024)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))


In [172]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (1, None, 256)            16640     
_________________________________________________________________
gru_16 (GRU)                 (1, None, 1024)           3938304   
_________________________________________________________________
dense_16 (Dense)             (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [173]:
def generate_text(model, start_string):

    # number of characters to generate
    text_length = 1000

    # vectorize the start_string
    input_sequence = [char2idx[c] for c in start_string]
    input_sequence = tf.expand_dims(input_sequence, 0)

    text_generated = []
    # some comment related to temparature
    temparature = 1.0

    model.reset_states()
    for i in range(text_length):
        predictions = model(input_sequence)

        # removing the batch dimension since the batch size == 1
        predictions = tf.squeeze(predictions, 0)

        # taking out the last predicted character in the output sequence
        predicted_id = tf.random.categorical( predictions, num_samples=1)[-1,0].numpy()
        
        input_sequence = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])    

    return (start_string + "".join(text_generated))

In [174]:
print(generate_text(model, u"ROMEO: "))

ROMEO: I say not to bring the king
Is wise another ancertaring: cracks the proud cease.
Being boxemed, the most get your brothers both;
Without your patience, to myself; for that I were she
standing my way, I say. I have seen.

SEBASTIAN:
Ay, a thousand lives and the hand were broke.
Your mother, would he fear us none so well at once;' any
'Tis he at your house they be diered at once than gone:
And when I do now, I pray you.

Fetch all floods; id
hath power to crush. Nay, they are rising,
For he did bear my country's face.
HatHARD IV:
Take the usly, and every thing about thee; therein desire
It is my wife, that once he perceives the body,
For stone together to our arrival froward,
As is the hand, though Marcius is proud;
And in the hand of death!
Look, and bleased, say these must be colighbe,
And wretched me brother, the rinds are husbany:
Since you, my noble cousix to the queen,
Devouring our readiness in revivered.

Second Murderer:
A matter, none, but of and waiting, who
Do girth, a